### Import the libraries

In [ ]:
from astropy.time import Time
from astroquery.jplhorizons import Horizons

import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import requests
import time

### Avoid repeating asteorids

Sometimes, during the Horizons query, some problems may appear. For example, bad internet connection. If we restart the code,
we don't want to repeat the process for asteroids that already have a .csv file. This part of the code prevent that.

In [ ]:
#EXTRA
#Specify the folder path where your CSV files are located
folder_path = '/home/milagros/Documents/run-3bis'

# Initialize an empty list to store the formatted asteroid numbers
formatted_asteroid_numbers = []

# Define a regular expression pattern to match the filenames
pattern = r'ATLAS-asteroid(\d+)\.csv'

# List all files in the folder
files = os.listdir(folder_path)

# Iterate through the files and extract the numbers
for filename in files:
    match = re.match(pattern, filename)
    if match:
        # Extract the digits from the matched pattern
        number = match.group(1)
        
        # Ensure a length of 5 with leading zeros
        formatted_number = number.zfill(6)
        
        formatted_asteroid_numbers.append(formatted_number)

### Read the catalog

In [ ]:
# Dataframe
df = pd.read_table('atlas-sscat.v1.0.dat', skiprows=15520305, nrows=7759205, header=None, delimiter=' ', dtype={15: 'string'},
                   usecols=[0,2,3,4,7,8,9,15])
df = df.rename(columns={0:'MJD', 2:'err_m', 3:'Filter', 4:'m', 7:'delta',
                  8:'r', 9:'alpha', 15:'idAsteroid'})
df = df[df["idAsteroid"].str.contains("[a-zA-Z]") == True] # Here we select only asteroids with packed designations
df = df[['idAsteroid', 'MJD', 'm', 'err_m', 'alpha','Filter', 'delta', 'r']]

### Distinguish between permanent packed designation and provisional packed designation

In [ ]:
def unpack_packed_designation(packed_des):
    """
    Unpacks a packed permanent designation as described in
    https://www.minorplanetcenter.net/iau/info/PackedDes.html

    Args:
        packed_des (str): Packed permanent designation.

    Returns:
        str: Unpacked permanent designation.
    """
    # Check if the packed designation meets the criteria
    if len(packed_des) == 5 and not packed_des.isnumeric() and packed_des[1:].isnumeric():
        # Extract components
        number = int(packed_des[1:])
        first_char = packed_des[0]

        # Convert first character to a numeric value
        if first_char.islower():
            first_val = ord(first_char) - ord('a') + 36
        elif first_char.isupper():
            first_val = ord(first_char) - ord('A') + 10
        else:
            # Handle the case when the first character isn't alphanumeric
            raise ValueError("Invalid first character in packed designation")

        # Calculate the unpacked number
        unpacked_number = number + first_val * 10000

        return str(unpacked_number)

    else:
        # Handle the case when the packed designation doesn't meet the criteria
        invalid_packed_designations.append(packed_des)
        return (np.nan)

In [ ]:
# Here we make a list of provisional packed designations and comets and delete them from the dataframe
invalid_packed_designations = []
df['idAsteroid'] = df['idAsteroid'].apply(unpack_packed_designation)
mylist = list(dict.fromkeys(invalid_packed_designations))
df = df.dropna()

In [ ]:
df = df[~df['idAsteroid'].isin(formatted_asteroid_numbers)]

In [ ]:
# We need to perform some calculations in order to obtain the desired format for the table.

# Compute reduced magnitude
m_red = df['m'] - 2.5*np.log10(df['r']*df['delta'])
m_red = m_red.to_numpy()
df.insert(5, "m_red", m_red, True)

# Compute flux
flux = 10**(-0.4*df.m)
flux = flux.to_numpy()
df.insert(4, "flux", flux, True)
e_flux = 0.4*10**(-0.4*df.m)*df.err_m
df.insert(5, "err_flux", e_flux, True)

# Compute JD
jd = df['MJD'] + 2400000.5
df.insert(2, "jd", jd, True)

#Add ephemeris columns
#df.insert(1, 'Desig', '')
df.insert(6, 'ElongFlag', '')

#Add constant columns
df.insert(7, 'Source', 'ATLAS')
df.insert(8, 'TypePhotometry', 'relative')
df.insert(9, 'LTcorrected', 1)
# -----------------------------------------------------

### Splitting dataframe

We have a dataframe with all the asteroids. For the ephemeris step, we need to proceed asteroid by asteroid.

In [ ]:
# Splitting dataframes
df = df[['idAsteroid', 'jd', 'm', 'err_m', 'flux', 'err_flux', 'm_red','Filter', 'alpha', 'delta', 'r', 'ElongFlag', 'Source', 'TypePhotometry', 'LTcorrected']]
sample = df
ids_sample = sample['idAsteroid'].drop_duplicates()
ids_sample = ids_sample.to_numpy()
num_ids = ids_sample.size  # Rename 'len' to 'num_ids'
print('Splitting...')

# Function to split DataFrame based on the changing value in the 'target_column'
def split_dataframe_on_value_change(df, column_name):
    df_list = []
    prev_value = None
    temp_df = None

    for index, row in df.iterrows():
        current_value = row[column_name]

        if prev_value is None or current_value != prev_value:
            if temp_df is not None:
                df_list.append(temp_df)
            temp_df = pd.DataFrame(columns=df.columns)

        temp_df = pd.concat([temp_df, row.to_frame().T], ignore_index=True)
        prev_value = current_value

    if temp_df is not None:
        df_list.append(temp_df)

    return df_list

# Split the DataFrame based on the changing value in the 'target_column'
resulting_dataframes = split_dataframe_on_value_change(sample, 'idAsteroid')
print('Splitted')

### Ephemeris

In [ ]:
# Ephemeris
for idx, df in enumerate(resulting_dataframes):
    ids = df['idAsteroid'].drop_duplicates().to_numpy()[0]
    jd = df['jd'].to_numpy()

    # If the list we pass to Horizons contains more than 50 elements, it throws an error.
    if jd.size <= 50:
        obj = Horizons(id=ids, location='399',
                epochs=jd, id_type='smallbody')

        #Ephemeris query
        eph = obj.ephemerides()
        elong_flag = eph.columns['elongFlag']

        #Vectors query
        vec = obj.vectors()
        x_obs = vec.columns['x']
        y_obs = vec.columns['y']
        z_obs = vec.columns['z']
        LT_obs = vec.columns['lighttime']

        #Query Sun-----------------------------------------
        jd_tdb = Time(jd.tolist(), format='jd', scale='utc').tdb.value
        obj_sun = Horizons(id=ids, location='500@10',
                epochs=jd_tdb, id_type='smallbody')
        vec_sun = obj.vectors()
        x_sun = vec_sun.columns['x']
        y_sun = vec_sun.columns['y']
        z_sun = vec_sun.columns['z']  

        #Put in dataframe

        df['ElongFlag'] = elong_flag
        df['x_obs'] = x_obs
        df['y_obs'] = y_obs
        df['z_obs'] = z_obs
        df['x_sun'] = x_sun
        df['y_sun'] = y_sun
        df['z_sun'] = z_sun
        df['LT_obs'] = LT_obs
        
        df.to_csv('/home/milagros/Documents/run-3bis/ATLAS-asteroid'+str(ids)+'.csv', index=False) 
        
        del obj
        del eph
        del obj_sun
        del vec_sun
        gc.collect()
        print(ids, 'small done')

    # To process lists of more than 50 elements, we split the dataframe and work part by part. At the end we merge everything into a single table.
    else:
        split_dataframes = np.array_split(df, 24)
        calculated_dataframes = []
        for i in split_dataframes:
            obj = Horizons(id=ids, location='399',epochs=i.jd.to_numpy(), id_type='smallbody')
            #Ephemeris query
            eph = obj.ephemerides()
            elong_flag = eph.columns['elongFlag']
            #Vectors query
            vec = obj.vectors()
            x_obs = vec.columns['x']
            y_obs = vec.columns['y']
            z_obs = vec.columns['z']
            LT_obs = vec.columns['lighttime']
            
            #Query Sun-----------------------------------------
            jd_tdb = Time(i.jd.tolist(), format='jd', scale='utc').tdb.value
            obj_sun = Horizons(id=ids, location='500@10',epochs=jd_tdb, id_type='smallbody')
            vec_sun = obj.vectors()
            x_sun = vec_sun.columns['x']
            y_sun = vec_sun.columns['y']
            z_sun = vec_sun.columns['z']
            
            i['ElongFlag'] = elong_flag
            i['x_obs'] = x_obs
            i['y_obs'] = y_obs
            i['z_obs'] = z_obs
            i['x_sun'] = x_sun
            i['y_sun'] = y_sun
            i['z_sun'] = z_sun
            i['LT_obs'] = LT_obs

            calculated_dataframes.append(i)
            del obj
            del eph
            del obj_sun
            del vec_sun
            
        merged_df = pd.concat(calculated_dataframes, ignore_index=True)
        
        merged_df['ElongFlag'] = merged_df['ElongFlag'].to_numpy()
        merged_df['x_obs'] = merged_df['x_obs'].to_numpy()
        merged_df['y_obs'] = merged_df['y_obs'].to_numpy()
        merged_df['z_obs'] = merged_df['z_obs'].to_numpy()
        merged_df['x_sun'] = merged_df['x_sun'].to_numpy()
        merged_df['y_sun'] = merged_df['y_sun'].to_numpy()
        merged_df['z_sun'] = merged_df['z_sun'].to_numpy()
        merged_df['LT_obs'] = merged_df['LT_obs'].to_numpy()
        
        merged_df.to_csv('/home/milagros/Documents/run-3bis/ATLAS-asteroid'+str(ids)+'.csv', index=False)
        print(ids, 'big done')
    
        gc.collect()

print('Done')

The end :)